# Автодополнение текство: Сравнение LSTM и Transformer моделей

Проект по сравнению эффективности LSTM и Transformer архитектур для задачи генерации и автодополнения текста на основе уменьшенной версии датасета sentiment140.


### Важное примечание о данных

Датасет был сокращен вручную из-за ограничений вычислительных ресурсов:

Исходный датасет sentiment140 содержал 1.6 млн твитов
Использованная версия: ~38 тыс. примеров
Причина: "ядро умирало" при обработке полного датасета
Это повлияло на результаты, особенно на трансформерные модели, которые требуют больших объемов данных.



In [1]:
!pip install pandas -q
!pip install torch -q
!pip install scikit-learn -q
!pip install transformers -q
!pip install rouge-score -q
!pip install tqdm -q
import os
import re
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from transformers import BertTokenizerFast, AutoTokenizer, AutoModelForCausalLM
from rouge_score import rouge_scorer
from tqdm import tqdm

os.environ["TOKENIZERS_PARALLELISM"] = "false"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Используем устройство:", device)


/Users/maria.chuvakinamail.ru/miniforge3/envs/textauto/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Используем устройство: cpu


In [2]:
from src.data_utils import load_and_prepare_data
from src.next_token_dataset import TextDataset
from src.lstm_model import LSTMModel

In [3]:
df, tokenizer, tokens = load_and_prepare_data(data_dir="/Users/maria.chuvakinamail.ru/text-autocomplete/data", max_len=20, min_len=7, use_sample=True)

print("Пример первых строк датасета:")
print(df.head())
print("Пример токенов:")
print(tokens['input_ids'][:2])


Пример первых строк датасета:
                                                text  num_tokens  \
0  switchfoot httptwitpiccom2y1zl awww thats a bu...          18   
1  is upset that he cant update his facebook by t...          21   
2  kenichan i dived many times for the ball manag...          18   
3     my whole body feels itchy and like its on fire          10   
4  nationwideclass no its not behaving at all im ...          21   

                                              tokens  
0  [switchfoot, httptwitpiccom2y1zl, awww, thats,...  
1  [is, upset, that, he, cant, update, his, faceb...  
2  [kenichan, i, dived, many, times, for, the, ba...  
3  [my, whole, body, feels, itchy, and, like, its...  
4  [nationwideclass, no, its, not, behaving, at, ...  
Пример токенов:
tensor([[  101,  6942, 13064,  8299,  2102,  9148, 25856,  2594,  9006,  2475,
          2100,  2487,  2480,  2140, 22091,  2860,  2860,  2008,  2015,   102],
        [  101,  2003,  6314,  2008,  2002,  2064,  2102

In [4]:
vocab_size = tokenizer.vocab_size
model = LSTMModel(vocab_size, embed_dim=128, hidden_dim=128, num_layers=1).to(device)
print(model)

LSTMModel(
  (embedding): Embedding(30522, 128)
  (lstm): LSTM(128, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=30522, bias=True)
)


In [5]:

def create_examples(tokens_ids):
    X_list, Y_list = [], []
    for token in tokens_ids:
        X = token[:-1]
        Y = token[1:]
        X_list.append(torch.tensor(X))
        Y_list.append(torch.tensor(Y))
    return X_list, Y_list

token_ids = tokens['input_ids'].tolist()
train_ids, temp_ids = train_test_split(token_ids, test_size=0.2, random_state=42)
val_ids, test_ids = train_test_split(temp_ids, test_size=0.5, random_state=42)

X_train, Y_train = create_examples(train_ids)


scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Функция генерации последовательности
def generate_sequence(model, start_seq, max_len=20):
    model.eval()
    generated = start_seq.tolist()
    input_seq = start_seq.unsqueeze(0).to(device)
    hidden = None
    for _ in range(max_len):
        logits, hidden = model(input_seq, hidden)
        next_token = torch.argmax(logits[:, -1, :], dim=-1)
        generated.append(next_token.item())
        input_seq = torch.cat([input_seq, next_token.unsqueeze(0)], dim=1)
    return generated

# Тренировка одной эпохи
def train_epoch(model, loader):
    model.train()
    total_loss = 0
    for X, Y in tqdm(loader):
        X, Y = X.to(device), Y.to(device)
        optimizer.zero_grad()
        logits, _ = model(X)
        loss = criterion(logits.view(-1, vocab_size), Y.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# Оценка ROUGE
def evaluate(model, loader):
    model.eval()
    scores = []
    with torch.no_grad():
        for X, Y in loader:
            X, Y = X.to(device), Y.to(device)
            for i in range(X.size(0)):
                seq_input = X[i][:int(0.75*X.size(1))]
                target_seq = Y[i][int(0.75*Y.size(1)):]
                pred_tokens = generate_sequence(model, seq_input, max_len=len(target_seq))
                pred_text = tokenizer.decode(pred_tokens[-len(target_seq):])
                target_text = tokenizer.decode(target_seq.tolist())
                score = scorer.score(pred_text, target_text)['rougeL'].fmeasure
                scores.append(score)
    return sum(scores)/len(scores)

X_val, Y_val = create_examples(val_ids)
X_test, Y_test = create_examples(test_ids)

train_dataset = TextDataset(X_train, Y_train)
val_dataset = TextDataset(X_val, Y_val)
test_dataset = TextDataset(X_test, Y_test)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

print("Размеры выборок:")
print('Размер тренирововчной выборки: ', len(train_dataset))
print('Размер валидационной выборки: ', len(val_dataset))
print('Размер обучающей выборки: ', len(test_dataset))


Размеры выборок:
Размер тренирововчной выборки:  330
Размер валидационной выборки:  41
Размер обучающей выборки:  42


In [6]:

# Этап 3: Тренировка LSTM

for epoch in range(2):
    loss = train_epoch(model, train_loader)
    rouge_score = evaluate(model, val_loader)
    print(f"Epoch {epoch+1}: Loss={loss:.4f}, ROUGE-L={rouge_score:.4f}")


  0%|          | 0/21 [00:00<?, ?it/s]/Users/maria.chuvakinamail.ru/text-autocomplete/src/next_token_dataset.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(self.X[idx]), torch.tensor(self.Y[idx])
100%|██████████| 21/21 [00:01<00:00, 12.50it/s]


Epoch 1: Loss=9.9468, ROUGE-L=0.3669


100%|██████████| 21/21 [00:01<00:00, 15.25it/s]


Epoch 2: Loss=7.4674, ROUGE-L=0.3463


In [7]:
# Этап 4: Предобученный трансформер
tokenizer_gpt = AutoTokenizer.from_pretrained("distilgpt2")
model_gpt = AutoModelForCausalLM.from_pretrained("distilgpt2").to(device)

def generate_gpt2(text, max_new_tokens=20):
    inputs = tokenizer_gpt(text, return_tensors="pt").to(device)
    outputs = model_gpt.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.8,
        pad_token_id=tokenizer_gpt.eos_token_id  # важно для GPT2
    )
    return tokenizer_gpt.decode(outputs[0], skip_special_tokens=True)

scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
# Проверка на одном примере 
text = df['text'][0]
input_text = ' '.join(text.split()[:int(len(text.split())*0.75)])
target_text = ' '.join(text.split()[int(len(text.split())*0.75):])
pred_text = generate_gpt2(input_text)
score = scorer.score(pred_text, target_text)['rougeL'].fmeasure

print("Input:", input_text)
print("Target:", target_text)
print("Prediction:", pred_text)
print("ROUGE-L score:", score)

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 3c43f205-8c08-441c-90d6-2cc633041d32)')' thrown while requesting HEAD https://huggingface.co/distilgpt2/resolve/main/generation_config.json
Retrying in 1s [Retry 1/5].
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 8f5bb950-bd3f-4a8c-bc8d-7bb723a9563b)')' thrown while requesting HEAD https://huggingface.co/distilgpt2/resolve/main/generation_config.json
Retrying in 2s [Retry 2/5].
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 33a9bb74-2798-4d67-9ac4-6db2bcac3112)')' thrown while requesting HEAD https://huggingface.co/distilgpt2/resolve/main/generation_config.json
Retrying in 4s [Retry 3/5].
'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /distilgpt2/resol

KeyboardInterrupt: 

In [ ]:
scores = []
for text in df['text'][:5]:  # берем только 5 примеров
    input_text = ' '.join(text.split()[:int(len(text.split())*0.75)])
    target_text = ' '.join(text.split()[int(len(text.split())*0.75):])
    pred_text = generate_gpt2(input_text)
    score = scorer.score(pred_text, target_text)['rougeL'].fmeasure
    scores.append(score)

print("Average ROUGE-L for 5 examples:", sum(scores)/len(scores))

In [ ]:

# Этап 5: Выводы
print("Сравнение моделей:")
print("- LSTM: ROUGE-L на валидации (после 2 эпох):", rouge_score)
print("- DistilGPT2: ROUGE-L на первых 50 примерах:", sum(scores)/len(scores))


Тк запускаю локально, к сожалению, из-за вычислительных ресурсов не удается проверить метрики качества, но прилагаю результаты, которые получились на ВМ (по какой-то причине ноутбук не сохранился)

**Результаты обучения на ВМ:**

- LSTM: 0.346	
- DistilGPT2: 0.045

**Сравнение моделей:**

LSTM:

- Быстрее обучение
- Меньше требований к памяти
- Стабильная сходимость
- Transformer пострадал от сокращения:


Transformer:
- Недостаточно данных
- Низкое качество
- Требует полного датасета для раскрытия потенциала
- Ключевые выводы

LSTM показала лучшие результаты на сокращенных данных:

Высокое качество (ROUGE-L: 0.346 vs 0.045) Эффективность на малых данных - главное преимущество Быстрая сходимость - хорошие результаты после 2 эпох Устойчивость к ограничениям ресурсов